In [1]:
# Run this cell for autoreload of modules
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
from breastcatt.fvit import multimodal_vit_base_patch16

/home/guillermo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = multimodal_vit_base_patch16(num_classes=1)
print(model)

MultiModalVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 1024, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-7): 8 x MultiModalBlock(
      (block): Block(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (act): GELU(approximate='none')
          (drop1): Dropout(p=0.0, inpla

In [4]:
from torchvision import transforms

data_transform = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor()
])

In [5]:
from datasets import load_dataset

dataset = load_dataset("SemilleroCV/BreastThermography")
sample = dataset['test'][0]

In [6]:
import numpy as np
from PIL import Image

img = sample['image']
img = np.array(img, dtype=np.float32)
txt = sample['text']

MAX_TEMPERATURE = 36.44
matrix = img / MAX_TEMPERATURE #normalizar
matrix = Image.fromarray(matrix) # np a pil image
matrix = data_transform(matrix) # trasnformaciones
matrix  = matrix.unsqueeze(0)
print(matrix.shape) # (1, 1, 224, 224)

torch.Size([1, 1, 224, 224])


In [7]:
from transformers import AutoModel, AutoTokenizer, AutoConfig

tokenizer= AutoTokenizer.from_pretrained('UFNLP/gatortron-base')
config=AutoConfig.from_pretrained('UFNLP/gatortron-base')
model_lm=AutoModel.from_pretrained('UFNLP/gatortron-base')
# model = model.to(device)

In [8]:
inputs = tokenizer(txt, return_tensors="pt")
outputs = model_lm(**inputs)
# Assume model returns a 'pooler_output' attribute containing the embeddings
features = outputs.pooler_output
features = features.unsqueeze(0)
print(features.shape) 

torch.Size([1, 1, 1024])


In [9]:
output = model(matrix, text_embedding=features)

In [10]:
output.shape

torch.Size([1, 1])